In [144]:
import pandas as pd
import numpy as np

In [145]:
df = pd.read_csv('changed_debt_rating.csv').drop(['Unnamed: 0'], axis = 1)
df['datadate'] = pd.to_datetime(df['datadate'])
df = df.set_index('datadate')
df = df.dropna(subset = ['splticrm'])

df['next_splticrm'] = df.groupby(['tic','gvkey'])['splticrm'].shift(-1)
df = df.dropna(subset = ['next_splticrm'])

print(len(df['gvkey'].unique()))
df.isna().sum()

11


gvkey            0
splticrm         0
busdesc          0
ggroup           0
gsector          0
conm             0
tic              0
next_splticrm    0
dtype: int64

In [146]:
df[df['splticrm'] == 'BBB']['gvkey'].unique()

array([  7063,   8692,  15142,  27780,  30384,  30637, 174022])

In [147]:
df['splticrm'].value_counts()

BBB     568
BB      401
BBB-    317
BB+     232
B+      202
BB-     195
A-      173
BBB+     68
B        42
CC       30
CCC      28
D        17
B-       13
CCC+      1
CCC-      1
Name: splticrm, dtype: int64

In [148]:
grouped_gv = df.groupby(['tic','gvkey'])['splticrm'].unique().reset_index()
grouped_gv

,tic,gvkey,splticrm
0,BPYU,28119,"[BBB-, BB+, BB, CCC+, CCC-, CC, D]"
1,FR,30384,"[BBB, BBB-, BB, BB-, B+, BB+]"
2,HST,7063,"[A-, BBB, BB-, BB, B+, BB+]"
3,NHI,24533,"[BBB-, BB, B+, BB-]"
4,OHI,25633,"[BBB-, BB+, BB-, B+, B, BB]"
5,PCH,8692,"[BBB+, A-, BBB, BBB-, BB+, BB]"
6,RLGY,174022,"[BBB, BB+, B+, B, B-, CC, CCC, BB-]"
7,SITC,27780,"[BBB, BBB-, BB, BB+]"
8,STAR,15142,"[BBB-, BBB, BB, B-, CCC, B+]"
9,VER,186783,"[BBB-, BB, BB+]"


In [149]:
# proj_list = [7063, 8692, 15142, 24533, 25633, 27780, 28119, 30384, 30637, 174022, 186783]
# concentrate = df[df['gvkey'].apply(lambda x: x in proj_list)]

proj_list = grouped_gv.tic.to_list()
concentrate = df[df['tic'].apply(lambda x: x in proj_list)]
concentrate

,gvkey,splticrm,busdesc,ggroup,gsector,conm,tic,next_splticrm
datadate,,,,,,,,
1985-12-31,7063,A-,"Host Hotels & Resorts, Inc. is an S&P 500 comp...",6010,60,HOST HOTELS & RESORTS INC,HST,A-
1986-01-31,7063,A-,"Host Hotels & Resorts, Inc. is an S&P 500 comp...",6010,60,HOST HOTELS & RESORTS INC,HST,A-
1986-02-28,7063,A-,"Host Hotels & Resorts, Inc. is an S&P 500 comp...",6010,60,HOST HOTELS & RESORTS INC,HST,A-
1986-03-31,7063,A-,"Host Hotels & Resorts, Inc. is an S&P 500 comp...",6010,60,HOST HOTELS & RESORTS INC,HST,A-
1986-04-30,7063,A-,"Host Hotels & Resorts, Inc. is an S&P 500 comp...",6010,60,HOST HOTELS & RESORTS INC,HST,A-
...,...,...,...,...,...,...,...,...
2016-09-30,186783,BB+,VEREIT is a full-service real estate operating...,6010,60,VEREIT INC,VER,BB+
2016-10-31,186783,BB+,VEREIT is a full-service real estate operating...,6010,60,VEREIT INC,VER,BB+
2016-11-30,186783,BB+,VEREIT is a full-service real estate operating...,6010,60,VEREIT INC,VER,BB+


In [194]:
def is_invest_grade(rating: str)->bool:
    return rating.startswith('A') or rating in {'BBB','BBB+'}

def junk_drop(rating:str, next_rating:str):
    return rating.apply(lambda x: is_invest_grade(x) == True) & next_rating.apply(lambda x: is_invest_grade(x) == False)


concentrate = concentrate[junk_drop(concentrate['splticrm'],concentrate['next_splticrm'])]
concentrate

,gvkey,splticrm,busdesc,ggroup,gsector,conm,tic,next_splticrm
datadate,,,,,,,,
1993-04-30,7063,BBB,"Host Hotels & Resorts, Inc. is an S&P 500 comp...",6010,60,HOST HOTELS & RESORTS INC,HST,BB-
2001-05-31,8692,BBB,PotlatchDeltic (NASDAQ:PCH) is a leading Real ...,6010,60,POTLATCHDELTIC CORP,PCH,BBB-
2008-09-30,15142,BBB,iStar Inc. (NYSE: STAR) is focused on reinvent...,6010,60,ISTAR INC,STAR,BBB-
2008-09-30,27780,BBB,SITE Centers is an owner and manager of open-a...,6010,60,SITE CENTERS CORP,SITC,BBB-
2008-09-30,30384,BBB,"First Industrial Realty Trust, Inc. (NYSE: FR)...",6010,60,FIRST INDL REALTY TRUST INC,FR,BBB-
2014-03-31,30637,BBB,"Veris Residential, Inc.",6010,60,MACK-CALI REALTY CORP,VRE,BBB-
2006-11-30,174022,BBB,"Realogy Holdings Corp., through its subsidiari...",6010,60,REALOGY HOLDINGS CORP,RLGY,BB+


In [195]:
# with open('fallen_angels.txt','w') as f:
#     f.write('\r'.join(set(concentrate.apply(lambda x: (str(x['gvkey'])+', '+str(x['tic'])), axis = 1))))

# concentrate['tic'].unique()

In [92]:
concentrate[['gvkey','tic']].drop_duplicates(subset=['gvkey','tic']).to_csv('fallen_angels.csv')